In [3]:
import pandas as pd

# 🔹 Load your combined ACS file
file_path = "/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv"
df = pd.read_csv(file_path)

# --- Clean up ---
df.columns = df.columns.str.strip().str.lower()
df = df.dropna(subset=["wavelength"])  # remove empty wavelength rows
df["wavelength"] = pd.to_numeric(df["wavelength"], errors="coerce")
df = df.dropna(subset=["wavelength"])
df = df.sort_values(["filename", "wavelength"])

# --- Identify parameters present ---
params = [p for p in ["agp", "cgp", "ap", "ag"] if p in df.columns]

# --- Build transposed (pivoted) tables for each parameter ---
pivoted_dfs = []
for param in params:
    pivot = df.pivot_table(index=["filename"], columns="wavelength", values=param)
    pivot.columns = [f"{param}_{int(c)}" for c in pivot.columns]
    pivoted_dfs.append(pivot)

# --- Merge all parameter tables horizontally ---
combined = pd.concat(pivoted_dfs, axis=1).reset_index()

# --- (Optional) add cruise if present ---
if "cruise" in df.columns:
    cruise_map = df.drop_duplicates(subset=["filename"])[["filename", "cruise"]]
    combined = combined.merge(cruise_map, on="filename", how="left")

# --- Reorder columns ---
cols = ["cruise", "filename"] + [c for c in combined.columns if c not in ["cruise", "filename"]]
combined = combined[cols]

# --- Save the final transposed DataFrame ---
output_path = "/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_transposed.csv"
combined.to_csv(output_path, index=False)

print(f"✅ Transposed file saved to:\n{output_path}")
print(f"Columns created: {len(combined.columns)} | Rows: {len(combined)}")


KeyError: 'filename'

In [5]:
import pandas as pd

# 🔹 Load your combined ACS file
file_path = "/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv"
df = pd.read_csv(file_path)

# --- Clean column names ---
df.columns = df.columns.str.strip().str.lower()

# --- Ensure wavelength is numeric ---
if "wavelength" not in df.columns:
    raise ValueError("❌ 'wavelength' column not found in file — check your CSV headers.")

df["wavelength"] = pd.to_numeric(df["wavelength"], errors="coerce")
df = df.dropna(subset=["wavelength"])

# --- If 'filename' not in columns, create a placeholder ---
if "filename" not in df.columns:
    df["filename"] = "unknown_file"

# --- If 'cruise' not in columns, add it too (optional) ---
if "cruise" not in df.columns:
    df["cruise"] = "unknown_cruise"

# --- Sort safely ---
df = df.sort_values(["filename", "wavelength"])

# --- Identify optical parameters present ---
params = [p for p in ["agp", "cgp", "ap", "ag"] if p in df.columns]

# --- Build transposed (pivoted) tables for each parameter ---
pivoted_dfs = []
for param in params:
    pivot = df.pivot_table(index=["filename"], columns="wavelength", values=param)
    pivot.columns = [f"{param}_{int(c)}" for c in pivot.columns]
    pivoted_dfs.append(pivot)

# --- Merge all parameter tables ---
combined = pd.concat(pivoted_dfs, axis=1).reset_index()

# --- Add cruise info back in ---
cruise_map = df.drop_duplicates(subset=["filename"])[["filename", "cruise"]]
combined = combined.merge(cruise_map, on="filename", how="left")

# --- Reorder columns ---
cols = ["cruise", "filename"] + [c for c in combined.columns if c not in ["cruise", "filename"]]
combined = combined[cols]

# --- Save the transposed DataFrame ---
output_path = "/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_transposed.csv"
combined.to_csv(output_path, index=False)

print(f"✅ Transposed file saved successfully at:\n{output_path}")
print(f"Columns created: {len(combined.columns)} | Rows: {len(combined)}")


✅ Transposed file saved successfully at:
/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_transposed.csv
Columns created: 1206 | Rows: 1


In [6]:
import pandas as pd

# 🔹 Load your CSV
file_path = "/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv"
df = pd.read_csv(file_path)

# --- Clean and prepare ---
df.columns = df.columns.str.strip().str.lower()

# Keep only relevant columns
cols = [c for c in ["wavelength", "agp", "cgp", "ap", "ag"] if c in df.columns]
df = df[cols]

# Drop NaNs and ensure numeric wavelength
df = df.dropna(subset=["wavelength"])
df["wavelength"] = pd.to_numeric(df["wavelength"], errors="coerce")
df = df.dropna(subset=["wavelength"])

# --- Transpose ---
# Set wavelength as columns, parameter names as rows
df_t = df.set_index("wavelength").T

# Optional: make wavelength column headers integers
df_t.columns = df_t.columns.astype(int)

# --- Save the transposed version ---
output_path = "/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_transposed.csv"
df_t.to_csv(output_path)

print(f"✅ Transposed wavelengths saved to:\n{output_path}")
print(f"Shape: {df_t.shape}")
print(df_t.head())


✅ Transposed wavelengths saved to:
/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_transposed.csv
Shape: (4, 11739)
wavelength       400       401       402       403       404       405  \
agp         0.077334  0.077756  0.078085  0.077690  0.078085  0.078085   
cgp         0.219700  0.218710  0.217730  0.216390  0.215540  0.214750   
ap          0.008705  0.009381  0.010058  0.011752  0.012013  0.012232   
ag          0.069318  0.068140  0.066961  0.066425  0.065853  0.065677   

wavelength       406       407       408       409  ...       691       692  \
agp         0.077690  0.077300  0.075613  0.074218  ...  0.010166  0.009759   
cgp         0.214200  0.214160  0.213750  0.213680  ...  0.133130  0.132740   
ap          0.014352  0.014638  0.014827  0.014827  ...  0.009278  0.008959   
ag          0.064252  0.061944  0.060974  0.058606  ...  0.000887  0.000800   

wavelength       693       694       695       696       697       698  \
agp         0.009488  0.009174 

In [7]:
import pandas as pd
from pathlib import Path

# 🔹 Input and output paths
input_path = Path("/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv")
output_path = input_path.parent / "wavelength_transposed_with_filename_cruise.csv"

# --- Load and clean ---
df = pd.read_csv(input_path)
df.columns = df.columns.str.strip().str.lower()

# --- Basic checks ---
if "wavelength" not in df.columns:
    raise ValueError("❌ No 'wavelength' column found in the dataset.")

param_cols = [c for c in ["agp", "cgp", "ap", "ag"] if c in df.columns]
if not param_cols:
    raise ValueError("❌ No optical parameter columns found (agp, cgp, ap, ag).")

# --- Handle wavelength values ---
df["wavelength"] = pd.to_numeric(df["wavelength"], errors="coerce")
df = df.dropna(subset=["wavelength"])

# --- Add filename and cruise columns if missing ---
if "filename" not in df.columns:
    df["filename"] = "unknown_file"

if "cruise" not in df.columns:
    # Try to infer cruise name from filename pattern, e.g. "CAH1609_xxx"
    df["cruise"] = df["filename"].str.extract(r"([A-Z]{3}\d{4})", expand=False)
    df["cruise"] = df["cruise"].fillna("unknown_cruise")

# --- Group by filename (and cruise) and transpose ---
output_list = []

for (fname, cruise), group in df.groupby(["filename", "cruise"]):
    # Keep wavelength + parameter columns
    sub = group[["wavelength"] + param_cols].dropna()
    sub = sub.set_index("wavelength").T  # Transpose wavelengths -> columns
    sub.columns = sub.columns.astype(int)  # Make wavelength integer columns

    # Add identifying info
    sub.insert(0, "parameter", sub.index)
    sub.insert(0, "filename", fname)
    sub.insert(0, "cruise", cruise)

    output_list.append(sub.reset_index(drop=True))

# --- Combine all and save ---
final_df = pd.concat(output_list, ignore_index=True)
final_df.to_csv(output_path, index=False)

print(f"✅ Transposed file saved at:\n{output_path}")
print("\n🔍 Preview of transposed table:\n")
print(final_df.head(10))


✅ Transposed file saved at:
/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/wavelength_transposed_with_filename_cruise.csv

🔍 Preview of transposed table:

wavelength          cruise      filename parameter       400       401  \
0           unknown_cruise  unknown_file       agp  0.077334  0.077756   
1           unknown_cruise  unknown_file       cgp  0.219700  0.218710   
2           unknown_cruise  unknown_file        ap  0.008705  0.009381   
3           unknown_cruise  unknown_file        ag  0.069318  0.068140   

wavelength       402       403       404       405       406  ...       691  \
0           0.078085  0.077690  0.078085  0.078085  0.077690  ...  0.010166   
1           0.217730  0.216390  0.215540  0.214750  0.214200  ...  0.133130   
2           0.010058  0.011752  0.012013  0.012232  0.014352  ...  0.009278   
3           0.066961  0.066425  0.065853  0.065677  0.064252  ...  0.000887   

wavelength       692       693       694       695       696       697  \


In [8]:
import pandas as pd
from pathlib import Path

# 🔹 Define the folder path
data_folder = Path("/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv")

# 🔹 List of specific files to process
files_to_process = ["CAH1609_31009_acs_R2.csv", "CAH1609_31015_acs_R2.csv", "CAH1609_31022_acs_R2", "CAH1609_31033_acs_R2", "CAH1609_31004_acs_R2", "CAH1609_31018_acs_R2","CAH1609_31001_acs_R2","CAH1609_31036_acs_R2","CAH1609_31027_acs_R2"]

# 🔹 Output path
output_path = data_folder / "CAH1609_transposed.csv"

# 🔹 Initialize list for all file results
transposed_blocks = []

for fname in files_to_process:
    fpath = data_folder / fname

    if not fpath.exists():
        print(f"⚠️ File not found: {fpath}")
        continue

    # --- Load CSV ---
    df = pd.read_csv(fpath)
    df.columns = df.columns.str.strip().str.lower()

    # --- Keep only needed columns ---
    keep_cols = [col for col in ["wavelength", "cgp", "agp", "ap", "ag"] if col in df.columns]
    if len(keep_cols) < 2:
        print(f"⚠️ Skipping {fname} — missing optical columns.")
        continue

    df = df[keep_cols].dropna(subset=["wavelength"])
    df["wavelength"] = pd.to_numeric(df["wavelength"], errors="coerce")
    df = df.dropna(subset=["wavelength"]).sort_values("wavelength")

    # --- Transpose wavelengths as columns ---
    df_t = df.set_index("wavelength").T
    df_t.columns = df_t.columns.astype(int)  # wavelength → int

    # --- Add metadata columns ---
    cruise = fname.split("_")[0]  # Extract cruise name like "CAH1609"
    df_t.insert(0, "parameter", df_t.index)
    df_t.insert(0, "filename", fname)
    df_t.insert(0, "cruise", cruise)

    transposed_blocks.append(df_t.reset_index(drop=True))

# --- Combine all results ---
if transposed_blocks:
    final_df = pd.concat(transposed_blocks, ignore_index=True)
    final_df.to_csv(output_path, index=False)
    print(f"✅ Transposed dataset saved to:\n{output_path}")
    print("\n🔍 Preview:")
    print(final_df.head(10))
else:
    print("❌ No valid data blocks found to transpose.")


⚠️ File not found: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/CAH1609_31009_acs_R2.csv
⚠️ File not found: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/CAH1609_31015_acs_R2.csv
⚠️ File not found: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/CAH1609_31022_acs_R2
⚠️ File not found: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/CAH1609_31033_acs_R2
⚠️ File not found: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/CAH1609_31004_acs_R2
⚠️ File not found: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/CAH1609_31018_acs_R2
⚠️ File not found: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/CAH1609_31001_acs_R2
⚠️ File not found: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/CAH1609_31036_acs_R2
⚠️ File not found: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/CAH1609_31027_acs_R2
❌ No valid data bloc

In [9]:
import pandas as pd
from pathlib import Path

# 🔹 Define your main path (file or folder)
data_path = Path("/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv")

# --- Detect if the path is a file or a folder ---
if data_path.is_file():
    data_folder = data_path.parent
else:
    data_folder = data_path

# 🔹 Automatically find all *_acs_R2* files in this folder
files_to_process = list(data_folder.glob("*_acs_R2*.csv"))

# If no CSVs found, let’s log it
if not files_to_process:
    print(f"⚠️ No '_acs_R2' CSV files found in {data_folder}")
else:
    print(f"🔹 Found {len(files_to_process)} files to process.")

# 🔹 Output path
output_path = data_folder / "CAH1609_transposed.csv"

# 🔹 Initialize list for all file results
transposed_blocks = []

for fpath in files_to_process:
    fname = fpath.name
    print(f"Processing → {fname}")

    try:
        df = pd.read_csv(fpath)
        df.columns = df.columns.str.strip().str.lower()
    except Exception as e:
        print(f"⚠️ Could not read {fname}: {e}")
        continue

    # --- Keep only needed columns ---
    keep_cols = [col for col in ["wavelength", "cgp", "agp", "ap", "ag"] if col in df.columns]
    if len(keep_cols) < 2:
        print(f"⚠️ Skipping {fname} — missing optical columns.")
        continue

    df = df[keep_cols].dropna(subset=["wavelength"])
    df["wavelength"] = pd.to_numeric(df["wavelength"], errors="coerce")
    df = df.dropna(subset=["wavelength"]).sort_values("wavelength")

    # --- Transpose wavelengths as columns ---
    df_t = df.set_index("wavelength").T
    df_t.columns = df_t.columns.astype(int)

    # --- Add metadata columns ---
    cruise = fname.split("_")[0]  # Extract cruise name like "CAH1609"
    df_t.insert(0, "parameter", df_t.index)
    df_t.insert(0, "filename", fname)
    df_t.insert(0, "cruise", cruise)

    transposed_blocks.append(df_t.reset_index(drop=True))

# --- Combine and export ---
if transposed_blocks:
    final_df = pd.concat(transposed_blocks, ignore_index=True)
    final_df.to_csv(output_path, index=False)
    print(f"✅ Transposed dataset saved to:\n{output_path}")
    print("\n🔍 Preview:")
    print(final_df.head(10))
else:
    print("❌ No valid data blocks found to transpose.")


⚠️ No '_acs_R2' CSV files found in /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609
❌ No valid data blocks found to transpose.


In [12]:
import pandas as pd
from pathlib import Path

# ✅ FIX: Point to the folder, not a file
base_folder = Path("/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv")

# 🔹 2. Find all "_acs_R2.csv" files recursively
acs_r2_files = [f for f in base_folder.rglob("*_acs_R2.csv")]

# 🔹 3. Create output folder for transposed results
output_folder = base_folder / "transposed_results"
output_folder.mkdir(exist_ok=True)

# 🔹 4. Process each file
for file in acs_r2_files:
    df = pd.read_csv(file)
    df.columns = [c.strip().lower() for c in df.columns]  # normalize column names
    
    required_cols = {'wavelength', 'agp', 'cgp', 'ag', 'ap'}
    if not required_cols.issubset(df.columns):
        print(f"⚠️ Skipping {file.name}: missing one of {required_cols}")
        continue
    
    # 🔹 5. Transpose so wavelength values become columns
    df_t = df.set_index('wavelength')[['agp', 'cgp', 'ag', 'ap']].T
    
    # 🔹 6. Add the filename (without extension) as an identifying column
    df_t.insert(0, 'source_file', file.stem)
    
    # 🔹 7. Save transposed file (non-destructive)
    output_path = output_folder / f"{file.stem}_transposed.csv"
    df_t.to_csv(output_path, index=True)
    
    print(f"✅ Transposed and saved: {output_path.name}")


NotADirectoryError: [Errno 20] Not a directory: '/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/combined_acs.csv/transposed_results'

In [23]:
import pandas as pd
from pathlib import Path

# 🔹 1. Folder containing your ACS CSVs
folder = Path("/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/")

# 🔹 2. Find all "_acs_R2.csv" files
acs_r2_files = list(folder.rglob("*_acs_R2.csv"))

# 🔹 3. List to collect transposed DataFrames
all_transposed = []

# 🔹 4. Process each file
for file in acs_r2_files:
    df = pd.read_csv(file)
    df.columns = [c.strip().lower() for c in df.columns]  # normalize column names

    required_cols = {'wavelength', 'agp', 'cgp', 'ag', 'ap'}
    if not required_cols.issubset(df.columns):
        print(f"⚠️ Skipping {file.name}: missing one of {required_cols}")
        continue

    # 🔹 Sort by wavelength numerically
    df = df.sort_values(by='wavelength')

    # 🔹 Select only desired columns
    df_selected = df[['wavelength', 'cgp', 'agp', 'ag', 'ap']]

    # 🔹 Transpose: wavelengths become columns, rows = cgp, agp, ag, ap
    df_t = df_selected.set_index('wavelength').T

    # 🔹 Add source file name for identification
    df_t.insert(0, 'source_file', file.stem)

    # 🔹 Collect for master CSV
    all_transposed.append(df_t)

# 🔹 5. Merge all transposed DataFrames into one master DataFrame
if all_transposed:
    master_df = pd.concat(all_transposed, ignore_index=True)
    master_csv_path = folder / "ACS_R2_Transposed_Master.csv"
    master_df.to_csv(master_csv_path, index=False)
    print(f"✅ Master CSV created: {master_csv_path}")
else:
    print("⚠️ No valid ACS R2 files found to merge.")


✅ Master CSV created: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/ACS_R2_Transposed_Master.csv


In [26]:
import pandas as pd
from pathlib import Path

# 🔹 1. Folder containing your ACS CSVs
folder = Path("/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609")

# 🔹 2. Find all "_acs_R2.csv" files
acs_r2_files = list(folder.rglob("*_acs_R2.csv"))

# 🔹 3. List to collect transposed DataFrames
all_transposed = []

# 🔹 4. Process each file
for file in acs_r2_files:
    df = pd.read_csv(file)
    df.columns = [c.strip().lower() for c in df.columns]  # normalize column names

    required_cols = {'wavelength', 'agp', 'cgp', 'ag', 'ap'}
    if not required_cols.issubset(df.columns):
        print(f"⚠️ Skipping {file.name}: missing one of {required_cols}")
        continue

    # 🔹 Sort by wavelength numerically
    df = df.sort_values(by='wavelength')

    # 🔹 Transpose the desired columns
    df_values = df[['cgp', 'agp', 'ap', 'ag']].T
    df_values.columns = df['wavelength']  # set wavelengths as column headers

    # 🔹 Add source_file as first column
    df_values.insert(0, 'source_file', file.stem)

    # 🔹 Append to list
    all_transposed.append(df_values)

# 🔹 5. Combine all transposed DataFrames
if all_transposed:
    master_df = pd.concat(all_transposed, ignore_index=True)
    master_csv_path = folder / "ACS_R2_Transposed_Master.csv"
    master_df.to_csv(master_csv_path, index=False)
    print(f"✅ Master CSV created: {master_csv_path}")
else:
    print("⚠️ No valid ACS R2 files found to merge.")


✅ Master CSV created: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/ACS_R2_Transposed_Master.csv


In [44]:
import pandas as pd
from pathlib import Path

# 🔹 1. Folder containing your ACS CSVs
folder = Path("/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609")

# 🔹 2. Find all "_acs_R2.csv" files
acs_r2_files = list(folder.rglob("*_acs_R2.csv"))

# 🔹 3. List to collect transposed DataFrames
all_transposed = []

# 🔹 4. Process each file
for file in acs_r2_files:
    df = pd.read_csv(file)
    df.columns = [c.strip().lower() for c in df.columns]  # normalize column names

    required_cols = {'wavelength', 'agp', 'cgp', 'ag', 'ap'}
    if not required_cols.issubset(df.columns):
        print(f"⚠️ Skipping {file.name}: missing one of {required_cols}")
        continue

    # 🔹 Sort by wavelength numerically
    df = df.sort_values(by='wavelength')

    # 🔹 Transpose the desired columns
    df_values = df[['cgp', 'agp', 'ag', 'ap']].T
    df_values.columns = df['wavelength']  # set wavelengths as column headers

    # 🔹 Add source_file as first column
    df_values.insert(0, 'source_file', file.stem)

    # 🔹 Add parameter name as second column
    df_values.insert(1, 'parameter', df_values.index)

    # 🔹 Append to list
    all_transposed.append(df_values)

# 🔹 5. Combine all transposed DataFrames
if all_transposed:
    master_df = pd.concat(all_transposed, ignore_index=True)
    master_csv_path = folder / "ACS_R2_Transposed_Master.csv"
    master_df.to_csv(master_csv_path, index=False)
    print(f"✅ Master CSV created: {master_csv_path}")
else:
    print("⚠️ No valid ACS R2 files found to merge.")


✅ Master CSV created: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CAH1609/ACS_R2_Transposed_Master.csv


In [43]:
import pandas as pd
from pathlib import Path

# 🔹 1. Folder containing your VSF CSVs
folder = Path("/Users/tuba/Downloads/UCONN/Dierssen/CORAL/CPA1705")

# 🔹 2. Find all "_vsf_R2.csv" files
vsf_r2_files = list(folder.rglob("*_vsf_R2.csv"))

# 🔹 3. List to collect transposed DataFrames
all_transposed = []

# 🔹 4. Process each file
for file in vsf_r2_files:
    try:
        df = pd.read_csv(file)
        df.columns = [c.strip().lower() for c in df.columns]  # normalize column names

        # 🔹 Identify possible sorting & wavelength columns
        wavelength_col = 'wavelength' if 'wavelength' in df.columns else None
        serial_col = 'serial' if 'serial' in df.columns else None

        # 🔹 Required optical columns (bbp, bbp_bp)
        value_cols = [col for col in ['bbp', 'bbp_bp'] if col in df.columns]
        if not value_cols:
            print(f"⚠️ Skipping {file.name}: missing 'bbp' or 'bbp_bp' columns.")
            continue

        # 🔹 Sort by serial if available, otherwise by wavelength
        if serial_col:
            df = df.sort_values(by=serial_col)
        elif wavelength_col:
            df = df.sort_values(by=wavelength_col)
        else:
            print(f"⚠️ Skipping {file.name}: no serial or wavelength column found.")
            continue

        # 🔹 Use wavelength or serial for column headers
        if wavelength_col:
            col_headers = df[wavelength_col]
        else:
            col_headers = df[serial_col]

        # 🔹 Transpose selected columns
        df_values = df[value_cols].T
        df_values.columns = col_headers  # wavelength/serial values as headers

        # 🔹 Add identifying columns
        df_values.insert(0, 'source_file', file.stem)
        df_values.insert(1, 'parameter', df_values.index)

        # 🔹 Append to list
        all_transposed.append(df_values)

    except Exception as e:
        print(f"❌ Error processing {file.name}: {e}")

# 🔹 5. Combine all transposed DataFrames into one master file
if all_transposed:
    master_df = pd.concat(all_transposed, ignore_index=True)
    master_csv_path = folder / "VSF_R2_BBP_Transposed_Master.csv"
    master_df.to_csv(master_csv_path, index=False)
    print(f"✅ Master CSV created successfully: {master_csv_path}")
else:
    print("⚠️ No valid VSF R2 files found to merge.")


✅ Master CSV created successfully: /Users/tuba/Downloads/UCONN/Dierssen/CORAL/CPA1705/VSF_R2_BBP_Transposed_Master.csv
